In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
# Change working directory
import os
os.chdir("/kaggle/input/")

In [ ]:
# Load datasets
baseDirName = "techlabs-wtsa/"

trainData = pd.read_csv(baseDirName+"data_final.csv")

In [ ]:
print("TF version   : ", tf.__version__)
print("GPU available: ", tf.config.list_physical_devices('GPU'))
print("Keras version   : ", keras.__version__)

In [ ]:
trainData["imageName"] = [str(image) + ".jpg" for image in trainData["id"]]
trainData["idTarget"] = np.reshape(trainData["id"].to_numpy(),(-1,))

In [ ]:
trainData.head(5)

In [ ]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=[0.7,1.0],
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    dtype=None,
)

data_generator = data_generator.flow_from_dataframe(trainData,
                                                    directory = baseDirName + "images/",
                                                    target_size = (224,224),
                                                    x_col = "imageName",
                                                    y_col = "idTarget",
                                                    class_mode = 'raw')

In [ ]:
import tensorflow_addons as tfa 
f1 = tfa.metrics.F1Score(len(trainData["id"]), 'macro')

def build_model(num_classes):
    inputs = layers.Input(shape=(224, 224, 3))
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(len(trainData["id"]), activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-3)
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=[f1])
    return model

In [ ]:
model = build_model(num_classes=len(trainData["id"]))
print(model.summary())

In [ ]:
epochs = 10
hist = model.fit(data_generator, epochs=epochs, verbose=2)

In [ ]:
import matplotlib.pyplot as plt
print(hist)

def plot_hist(hist):
    plt.plot(hist.history["f1_score"])
    plt.plot(hist.history["val_f1_score"])
    plt.title("model f1")
    plt.ylabel("f1")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    
plot_hist(hist)

In [ ]:
def unfreeze_model(model, fr=-20, lr=3e-4):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[fr:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[f1])

unfreeze_model(model, fr=-20, lr=3e-4)

checkpoint_filepath = "model_best_first_unfreeze.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_f1_score', mode='max', save_best_only=True)

epochs = 15  
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks = [checkpoint], verbose=2)
plot_hist(hist)

In [ ]:
unfreeze_model(model, fr=0, lr=1e-4)

checkpoint_filepath = "model_best.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_f1_score', mode='max', save_best_only=True)

epochs = 25  
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks = [checkpoint], verbose=2)
plot_hist(hist)